In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go 
import plotly.express as px 
import seaborn as sns
import scipy.stats as stats
from math import sqrt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

<h1>First notebook</h1>

In this notebook I will try to answer a really simple question.
Who was fastest in answering the survey.

This might seem like a silly question to answer, and I agree!
But seeing as this is my first attempt it seem like a good start, and I will hopefully learn from writing this words and untill it is published. 
Any feedback is welcome! And please upvote if you feel like I deserve it :)

In [ ]:
df = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
df.shape

In [ ]:
df.head()

In [ ]:
#Removing top row
df_fin = df.iloc[1:,:]


In [ ]:
df_fin

This turned out to be way more difficult than I first imagined. I made some scatter plots which failed to render, and the ones that did looked terrible because of the spread in the data. I think now it might be because I don't really at this point have a clear picture of what I want to find out. I am leaving in my fumbling because I want this notebook to show my progress and process, or lack thereof. 

So here I am, thinking aloud with words. What I want to do is split the data into series for each country. I then want to see how well those countries does it by looking at averages, spread (and more? I'm still learning this as well). This is bloody difficult, but at least it's still fun :)

In [ ]:
#Want to know how many unique countries there are
countries = df_fin["Q3"]
countryList = []
for country in df_fin["Q3"]:
    if not country in countryList:
        countryList.append(country)

print(len(countryList))
        

So this approach did work. I wanted to know how many countries there are, and this did the job. 

I also wanted to make one series for each country, and each series containing each of the times. I don't at this point fully understand how, or why my attempts didn't work, so i continue struggling with this. 

In [ ]:
"""
i = 0
counter = 0
df_countryAndTimes = pd.DataFrame()

for country in df_fin["Q3"]:
    temp_array = []
    i = 0
    for temp_country in df_fin["Q3"]:
        if temp_country == country:
            time = int(df_fin.iloc[i,0])
            temp_array.append(time)
        i += 1
    df_temp = pd.DataFrame({country:temp_array})
    df_countryAndTimes = df_countryAndTimes.append(df_temp)

"""
  

 


This was kind of hillarious. This code ran for foreeever! And it made a dataframe, but something felt off about it.  It showed that out of about 20 000 answers, 3 million were from India. Trying again! But! I felt like I was on the right track, I just needed to fix ... something 

In [ ]:
#Make dataframe with countries and times, and also filter out times that are above 3600. If you spend more than one hour, it doesnt count. (Arbitrary limit is arbitrary)

countriesCompleted = []
df_countryAndTimes = pd.DataFrame()

for country in df_fin["Q3"]:
    if not country in countriesCompleted:
        countriesCompleted.append(country)
        temp_array = []
        i = 0
        for temp_country in df_fin["Q3"]:

            if temp_country == country:
                time = int(df_fin.iloc[i,0])
                if time < 3600:
                    temp_array.append(time)
            i += 1
        df_temp = pd.DataFrame({country:temp_array})
        df_countryAndTimes = df_countryAndTimes.append(df_temp)

df_countryAndTimes =pd.concat([pd.Series(df_countryAndTimes[c].dropna().values, name=c) for c in df_countryAndTimes], axis=1)

In [ ]:
df_countryAndTimes

Finally it worked! And wow! More than 25% of the answers were from India, cool! But this approach did work! Now to try and do something with the data.

In [ ]:
df_countryAndTimes.describe()

I see that there is a lot of variety in the data. I also notice a lot of outliers. I assume a lot of them are because people forgot about the survey, or got distracted? So I decide to set a max limit on 3600 seconds. If you use more than one hour, you are disqualified. Let's see if I can do that.

Narrator: "He did not."

So I wrote this text before I did the fix. I decided to disqualify the above 3600s when I generated the dataframe. It was a simple solution, but if anyone can show me a better way I would appreciate it. 

In [ ]:
df_ctmean = df_countryAndTimes.mean().sort_values(ascending = False)
pd.options.plotting.backend = "plotly"

# using Plotly Express via the Pandas backend
fig1 = df_ctmean.plot.barh(height=1000)
fig1.show()

And there we go! The fastest country is Ireland with an average value of 561. Now, I will try to plot the spread of each dataset. With 55 countries this might be messy, but maybe not? 

In [ ]:
for country in countryList:
    df_temp = df_countryAndTimes[country].dropna()
    h = np.asarray(df_temp)
    h = sorted(h)
 
    #use the scipy stats module to fit a normal distirbution with same mean and standard deviation
    fit = stats.norm.pdf(h, np.mean(h), np.std(h)) 
    

    #plot both series on the histogram
    plt.plot(h,fit,'-',linewidth = 2,label="Normal distribution with same mean and var")
    plt.hist(h,density = True, bins = 50,label="Actual distribution")  
    plt.xlabel(country)
    plt.legend()
    plt.show()

I did it! It isn't very pretty, but I managed to plot every single country with a normal distribution with same values.

From this I can see that India has most of their times around 500 seconds, or around 8 minutes. I can also see that China has a lot of answers which take more than 1000 seconds, or about 16 minutes. Does this mean that the chinese are slower? 

Other countries that show a lot of answers above 1000 seconds are:
 Canada
 Switzerland
 South Africa
 Egypt
 Poland
 United Arab Emirates
 Ireland
 
 
I notice the distribution on Sri Lanka and Romania was kinda broken around the end. I imagine this is a result of the outliers in the data. 

In [ ]:
#Make dataframe with countries and times, this time with every time.

countriesCompleted = []
df_countryAndTimesWithAllData = pd.DataFrame()

for country in df_fin["Q3"]:
    if not country in countriesCompleted:
        countriesCompleted.append(country)
        temp_array = []
        i = 0
        for temp_country in df_fin["Q3"]:

            if temp_country == country:
                time = int(df_fin.iloc[i,0])
                temp_array.append(time)
            i += 1
        df_temp = pd.DataFrame({country:temp_array})
        df_countryAndTimesWithAllData = df_countryAndTimesWithAllData.append(df_temp)

df_countryAndTimesWithAllData =pd.concat([pd.Series(df_countryAndTimesWithAllData[c].dropna().values, name=c) for c in df_countryAndTimesWithAllData], axis=1)

In [ ]:
df_ctmean = df_countryAndTimesWithAllData.mean().sort_values(ascending = False)
pd.options.plotting.backend = "plotly"

# using Plotly Express via the Pandas backend
fig1 = df_ctmean.plot.barh(height=1000)
fig1.show()

In [ ]:
for country in countryList:
    df_temp = df_countryAndTimesWithAllData[country].dropna()
    h = np.asarray(df_temp)
    h = sorted(h)
 
    #use the scipy stats module to fit a normal distirbution with same mean and standard deviation
    fit = stats.norm.pdf(h, np.mean(h), np.std(h)) 
    

    #plot both series on the histogram
    plt.plot(h,fit,'-',linewidth = 2,label="Normal distribution with same mean and var")
    plt.hist(h,density = True, bins = 20,label="Actual distribution")  
    plt.xlabel(country)
    plt.legend()
    plt.show()

In [ ]:
chosenCountries = ["United States of America", "Republic of Korea", "Sweden"]
for country in chosenCountries:
    df_temp = df_countryAndTimes[country].dropna()
    h = np.asarray(df_temp)
    h = sorted(h)
 
    #use the scipy stats module to fit a normal distirbution with same mean and standard deviation
    fit = stats.norm.pdf(h, np.mean(h), np.std(h)) 
    

    #plot both series on the histogram
    plt.plot(h,fit,'-',linewidth = 2,label="Normal distribution with same mean and var")
    plt.hist(h,density = True, bins = 20,label="Actual distribution")  
    plt.xlabel(country)
    plt.legend()
    plt.show()
    
for country in chosenCountries:
    df_temp = df_countryAndTimesWithAllData[country].dropna()
    h = np.asarray(df_temp)
    h = sorted(h)
 
    #use the scipy stats module to fit a normal distirbution with same mean and standard deviation
    fit = stats.norm.pdf(h, np.mean(h), np.std(h)) 
    

    #plot both series on the histogram
    plt.plot(h,fit,'-',linewidth = 2,label="Normal distribution with same mean and var")
    plt.hist(h,density = True, bins = 20,label="Actual distribution")  
    plt.xlabel(country)
    plt.legend()
    plt.show()

This was kind of hillarious. I feel confident now that I should exclude the outliers. These plots look terrible! It least compared to the ones without the outliers. And the winner in average time now is sweden, where almost no participants used more than 4000 seconds. 

And this concludes my notebook. I learned a lot! 
I learned how I can extract data from dataframes, and get useful numerical data. 
I learned how to plot normal distributions and bar horisontal bar diagrams.
I learned that it is useful to exclude outliers. I still don't know where to set the line, so any feedback in that regards is very much appreciated. 

Congratulation Sweden and Ireland with winning the survey-time-competition that you didn't know you participated in. 